In [2]:
%load_ext autoreload
%autoreload 2
import logging
import sys, os
import talib
sys.path.insert(0, os.path.abspath(""+"../Crawler"))
sys.path.insert(0, os.path.abspath(""+"../Simulator"))

from Packing import Data_preprocessing
import Simulator
from Component import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# Simulator 讀取DATA
sim = Simulator.Simulator(updatePKL=False)

# Read pickle 
load origin pkl file.
load finance data pkl file.
Construct Components_lib.


In [4]:
# buy example
slow_k, slow_d = talib_Output(sim.DATA['台股個股'], talib.abstract.STOCH, {'fastk_period':9})
法人_buy = N日均線(sim.DATA['台股個股']['法人總計'], 3 ) > 500
成交張數大於2500張 = Component(sim.DATA['台股個股']['成交股數']/1000 > 2500)
buy = (slow_d < 10) & 成交張數大於2500張 & 法人_buy

# sell
sell_k = (slow_k > 85)
sell = buy.shift(10) | sell_k

In [6]:
# stocks data
sim.DATA['台股個股'].keys()

dict_keys(['自營商', '投信', 'High', '外資', '自營商(僅避險)', 'Close', 'Open', '法人總計', '成交金額', '本益比', '成交筆數', 'Volume', 'Low', '成交股數'])

In [7]:
# Use Mode - 'stocks'.
# Add new figure - 'Trade info', 
# which shows the stocks ID which have been sold & profit-rate ( = sell/buy - 1) & hold_info (Money)
sim.Run(buy, sell, Mode = 'stocks')

# -----------------------RESULTS------------------------ #
#                                                        #
#               Period =      2012-05-02 to 2019-01-24   #
#         Num of Trade =             303                 #
#                Money =               1                 #
#               Profit =          -0.393                 #
#           Max Profit =           0.058                 #
#           Min Profit =          -0.408                 #
# Return On Investment =          -39.3%                 #
#       Annualized ROI =          -5.05%                 #
#        Ave_hold_days =            9.16                 #
#        Max_hold_days =              10                 #
#        Min_hold_days =               1                 #
#                                                        #
# ------------------------------------------------------ #

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]



In [11]:
# sim._each_stocks_trade_count record the num of trades in each stocks 
# (Only keep StockID which have traded at least once)
sim._each_stocks_trade_count['1312'] # print num of trade

2.0

In [12]:
# plot StockID = 1312
# plot slow_k & slow_d together 
import plotly
import plotly.plotly as py

# settings
start_date = '2014-01-01'
end_date = '2019-01-01'
StockID = '1312'

fig = sim.Iplotly(start_date = start_date, end_date = end_date, Mode = 'Stocks', StockID = StockID, return_iplot = False)
slowk = slow_k.data[StockID].truncate(start_date, end_date, axis = 0)
slowd = slow_d.data[StockID].truncate(start_date, end_date, axis = 0)

# add slowk & slowd
fig['data'].append( dict(
        type = 'scatter',
        y = slowk,
        x = slowk.index,
        yaxis='y2',
        opacity = 0.5,
        name='slowk'
    ) )
fig['data'].append( dict(
        type = 'scatter',
        y = slowd,
        x = slowd.index,
        yaxis='y2',
        opacity = 0.5,
        name='slowd'
    ) )

fig['layout']['yaxis2'] = dict(
    range= [0, 500], 
    overlaying= 'y', 
    anchor= 'x', 
    side= 'right', 
)

# remove empty data date
fig['layout']['xaxis'] = {'type':'category'}

# Plot
py.iplot(fig, filename='TW_stocks')

# trade times: 2


In [15]:
# Reference
# 畫出每筆交易獲利分佈圖, (sell/buy - 1)
# Note: sim._Profit_rate_list 存每筆交易profit
# > 0, 交易獲利, < 0, 交易虧損
import plotly.graph_objs as go
data = [go.Histogram(x=sim._Profit_rate_list,
                     histnorm='probability')]
py.iplot(data, filename='normalized histogram')